<a href="https://colab.research.google.com/github/Group2-P6-Bachelor-Project/Model/blob/main/P6_Bachelor_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting and Importing packages

In [ ]:
!pip install pyunpack
!pip install patool
!pip uninstall tensorflow
!pip install tf-nightly
import tensorflow as tf
print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 1.3 MB/s eta 0:00:00
Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? 

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
from pyunpack import Archive

In [ ]:
!mkdir my_dataset
Archive('/content/gdrive/MyDrive/InitialV2.rar').extractall('/content/my_dataset')

In [ ]:
import tensorflow as tf
import pathlib

data_dir = pathlib.Path('/content/my_dataset/InitialV2')


# Attempt #2

In [ ]:
img_size = (260, 260)
batch_size = 32
train_accs = []
val_accs = []
seeds = [789]
for seed in seeds:
  train_ds, val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=0.2,
      subset= 'both',
      seed=seed,
      image_size=img_size,
      batch_size=batch_size)

  data_augmentation = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
      tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
      tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ])

  train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
  label_names = sorted(item.name for item in data_dir.glob('*/') if item.is_dir())
  label_encoder = tf.keras.layers.experimental.preprocessing.StringLookup()
  label_encoder.adapt(label_names)

  # Map the labels to integers
  train_labels = train_ds.map(lambda x, y: label_encoder(y))
  val_labels = val_ds.map(lambda x, y: label_encoder(y))

  num_classes = len(label_encoder.get_vocabulary())

  base_model = tf.keras.applications.EfficientNetB2(include_top=False, weights='imagenet', input_shape=(260, 260, 3))
  base_model.trainable = False
  inputs = tf.keras.Input(shape=(260, 260, 3))
  x = base_model(inputs, training=False)
  #Andres said it should be removed but removing it breaks the code
  x = tf.keras.layers.GlobalAveragePooling2D()(x)\
  #Add more layers her

  outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)

  #show model.summary

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  history = model.fit(train_ds,
                      validation_data=val_ds,
                      epochs=10)
  train_accs.append(history.history['accuracy'][-1])
  val_accs.append(history.history['val_accuracy'][-1])

for i in range(len(seeds)):
    print(f'Seed {seeds[i]}: Train accuracy = {train_accs[i]}, Validation accuracy = {val_accs[i]}')

In [ ]:
import matplotlib.pyplot as plt

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')

# Add labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')

# Show the legend
plt.legend()

# Display the plot
plt.show()

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

# Add labels and title
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

# Show the legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
#Confusion Matrix
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import itertools

# Get the true labels and predicted labels
y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_pred = np.argmax(model.predict(val_ds), axis=-1)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(val_ds.class_names))
plt.xticks(tick_marks, val_ds.class_names, rotation=45)
plt.yticks(tick_marks, val_ds.class_names)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# def plot_misclassified_images(val_test_ds, y_true, y_pred):
#     for i, (image, label) in enumerate(val_test_ds.unbatch()):
#         if y_true[i] != y_pred[i]:
#             plt.figure(figsize=(4,4))
#             plt.imshow(image.numpy().astype(np.uint8))
#             plt.title(f'True label: {val_test_ds.class_names[y_true[i]]}, Predicted label: {val_test_ds.class_names[y_pred[i]]}')
#             plt.axis('off')
#             plt.show()
# plot_misclassified_images(val_ds, y_true, y_pred)

In [ ]:
#Fine-Tuning add custom layers that fit our problem
#Freeze the base layers of the model and train only the custom layers that we added
#Unfreeze the top layers of the model and retrain them together with the custom layers
#Compare the results you are getting and modify the layers to get better results

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# # Get a batch of images and labels
# images, labels = next(iter(val_ds))
# image = tf.keras.preprocessing.image.load_img('/content/IMG_0947.jpg', target_size=(260, 260))

# image_array = tf.keras.preprocessing.image.img_to_array(image)
# image_array = tf.keras.applications.efficientnet.preprocess_input(image_array)
# image_tensor = tf.convert_to_tensor(image_array)
# image_tensor = tf.expand_dims(image_tensor, axis=0)
# # Make predictions on the batch of images
# predictions = model.predict(image_tensor)

# # Get the predicted classes
# predicted_class = np.argmax(predictions, axis=1)[0]
# predicted_probability = predictions[0][predicted_class]
# # Get the true classes
# class_names_copy = label_names.copy()
# predicted_name = class_names_copy.pop(predicted_class)

# # Get the accuracy
# other_probabilities = [predictions[0][i] for i in range(len(class_names_copy))]

# other_sum = sum(other_probabilities)

# # Normalize the probabilities of the other classes
# other_probabilities = [p / other_sum * (1 - predicted_probability) for p in other_probabilities]

# # Display the image and the predicted class
# img = images[0]
# class_names = sorted(os.listdir(data_dir))

# plt.imshow(image)
# plt.title(f'Predicted: {class_names[predicted_class]}, Accuracy: {predicted_probability:.2f}')
# plt.show()
# # Display the other classes and their probabilities
# for i in range(len(class_names_copy)):
#     print(f'{class_names_copy[i]}: {other_probabilities[i]:.2f}')

In [ ]:
model.save("model2.h5")

In [ ]:
# !zip -r /content/hundred_epoch_model.zip /content/model
# from google.colab import files
# files.download('/content/hundred_epoch_model.zip')

In [ ]:
# import tensorflow as tf

# tflite_output_path = '/content/converted_model.tflite'

# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
# tflite_model = converter.convert()

# with open(tflite_output_path, 'wb') as f:
#     f.write(tflite_model)

Testing tflite model

In [ ]:
# interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
# interpreter.allocate_tensors()

# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

In [ ]:
# import numpy as np
# from PIL import Image

# # Define the data augmentation pipeline
# data_augmentation = tf.keras.Sequential([
#   tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
#   tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
#   tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
# ])
# image = tf.keras.preprocessing.image.load_img('/content/IMG_0947.jpg', target_size=(260, 260))

# # Apply the data augmentation to the input image
# # image = data_augmentation(np.array(image))


# image = np.array(image, dtype=np.float32)


# image = tf.keras.applications.efficientnet.preprocess_input(image)
# image = np.expand_dims(image, 0)

In [ ]:
# interpreter.set_tensor(input_details[0]['index'], image)
# interpreter.invoke()
# output_data = interpreter.get_tensor(output_details[0]['index'])
# predicted_labels = np.argmax(output_data, axis=1)[0]

In [ ]:
# # Display the image and the predicted labels
# plt.imshow(image[0])
# plt.title("Predicted label: {}".format(class_names[predicted_labels]))
# plt.show()

# # Print the predicted probabilities for all labels
# for i in range(len(class_names)):
#     print("{}: {:.2f}%".format(class_names[i], output_data[0][i] * 100))

In [ ]:
# print(image[0].shape)

Validation with different datasets

In [ ]:
!mkdir my_validationset2

In [ ]:
from pyunpack import Archive
Archive('/content/gdrive/MyDrive/Validation_WastePicture.zip').extractall('/content/my_validationset2')

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
val_dir = pathlib.Path('/content/my_validationset2/Validation_WastePicture')
val_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
      val_dir,
      image_size=img_size,
      validation_split=0,
      batch_size=batch_size,
      shuffle=False)

In [ ]:
#Confusion Matrix
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import itertools

# Get the true labels and predicted labels
y_test_true = np.concatenate([y for x, y in val_test_ds], axis=0)
y_test_pred = np.argmax(model.predict(val_test_ds), axis=-1)

# Compute the confusion matrix
cm = confusion_matrix(y_test_true, y_test_pred)

# Plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(val_test_ds.class_names))
plt.xticks(tick_marks, val_test_ds.class_names, rotation=45)
plt.yticks(tick_marks, val_test_ds.class_names)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# plot_misclassified_images(val_test_ds, y_test_true, y_test_pred)

Testning 100 Epoch model with a confusion matrix

In [ ]:
# Archive('/content/gdrive/MyDrive/hundred_epoch_model.zip').extractall('/content/')
# saved_model_path = '/content/model2/'
# # Load the saved model
# loaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
# #Confusion Matrix
# import numpy as np
# import tensorflow as tf
# from sklearn.metrics import confusion_matrix
# import itertools

# # Get the true labels and predicted labels
# y_test_true = np.concatenate([y for x, y in val_test_ds], axis=0)
# y_test_pred = np.argmax(loaded_model.predict(val_test_ds), axis=-1)

# # Compute the confusion matrix
# cm = confusion_matrix(y_test_true, y_test_pred)

# # Plot the confusion matrix
# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.colorbar()
# tick_marks = np.arange(len(val_test_ds.class_names))
# plt.xticks(tick_marks, val_test_ds.class_names, rotation=45)
# plt.yticks(tick_marks, val_test_ds.class_names)

# thresh = cm.max() / 2.
# for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#     plt.text(j, i, format(cm[i, j], 'd'),
#              horizontalalignment="center",
#              color="white" if cm[i, j] > thresh else "black")

# plt.tight_layout()
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.show()

Ensembling

In [ ]:
model1 = tf.keras.models.load_model('model.h5')
model2 = tf.keras.models.load_model('model2.h5')
model3 = tf.keras.models.load_model('100_epoch.h5')

In [ ]:
import numpy as np

def ensemble_predict(models, dataset):
    predictions = []
    for model in models:
        predictions.append(model.predict(dataset))
    avg_predictions = np.mean(predictions, axis=0)
    return avg_predictions

ensemble_predictions = ensemble_predict([model1, model2, model3], val_test_ds)

In [ ]:
ground_truth_labels = np.concatenate([y for x, y in val_test_ds], axis=0)
# convert ensemble predictions to class labels
ensemble_labels = np.argmax(ensemble_predictions, axis=1)
ground_truth_labels = ground_truth_labels.astype('float32')

In [ ]:
print(ensemble_labels.shape, ground_truth_labels.shape)

In [ ]:
ensemble_metrics = model.evaluate(ensemble_labels, ground_truth_labels)